transferring columns from [master TML spreadsheet](https://wsponline.sharepoint.com/:x:/r/sites/WDS-ClimateResilienceandSustainabilityCanada/_layouts/15/Doc.aspx?sourcedoc=%7B0A099FC0-DDE6-4789-9A0B-3645CB9D93E4%7D&file=Master%20TML%20Listing%20Updated%202025%20-%20Climate%20Adaptation.xlsx&action=default&mobileredirect=true) and adding to [poles.csv](https://wsponline.sharepoint.com/:x:/r/sites/WDS-ClimateResilienceandSustainabilityCanada/Shared%20Documents/General/Projects/CA0060027.8940%20-%20NF%20Power%20CRiVA/Client%20Data/Extracted%20CSVs/Coastline%20v2%20with%20elevation%20and%20region/Poles.csv?d=w6109d5e6a3fa4d19bc788353664802c9&csf=1&web=1&e=rSMXuj)

column AV is Transmissi, contains line number column C in poles

In [16]:
import pandas as pd
import numpy as np

In [17]:
tmlf = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\NF Power GIS\Poles-tr\Master TML Listing Updated 2025 - Climate Adaptation.xlsx"
tml = pd.read_excel(tmlf, header = 3)
polesf = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\NF Power GIS\Coastline CSVs\Exports\v2 no interp with elev and reg\Poles.csv"
poles = pd.read_csv(polesf)

C:\Users\CAMG038492\AppData\Local\Temp\ipykernel_32164\930693666.py:4: DtypeWarning: Columns (11,12,15,21,22,34,37,39,43,46,47,48,50,51,53,54,55,67,69) have mixed types. Specify dtype option on import or set low_memory=False.
  poles = pd.read_csv(polesf)


In [18]:
filter = False
if filter:
    poles = poles.dropna(subset = 'Transmissi').copy(deep = True)
tml = tml.rename(columns = {"VOLTAGE\nkV": "VOLTAGE kV", "YEAR \nBUILT": "YEAR BUILT"})

In [19]:
poles.Transmissi.value_counts()

Transmissi
65L-1      4
124L-88    3
57L-45     3
416L-94    3
416L-85    3
          ..
51L-89     1
51L-110    1
73L-73     1
13L-48     1
24L-169    1
Name: count, Length: 7470, dtype: int64

In [20]:
def v_findline(row):
    string = row['Transmissi']
    if isinstance(string,str):
        pre = int(string.split("L")[0])
    elif np.isnan(string):
        pre = string
    else:
        pre = string
        print(f"{pre} unchanged")
    return pre

In [21]:
def v_findsub(row):
    string = row['Transmissi']
    if isinstance(string,str):
        pre = string.split("-")[1]
        try: pre = int(pre)
        except:
            try:
                number = int(''.join([char for char in pre if char.isdigit()]))
                pre = number
            except:
                pre = ""
    elif np.isnan(string):
        pre = ""
    else:
        pre = string
        print(f"{pre} unchanged")
    return pre

In [22]:
poles['Transmissi-pre'] = poles.apply(v_findline, axis = 1)
poles['Transmissi-sub'] = poles.apply(v_findsub, axis = 1)

In [23]:
poles[['Transmissi','Transmissi-pre','Transmissi-sub']].dropna()

,Transmissi,Transmissi-pre,Transmissi-sub
129187,12L-33,12.0,33
289127,57L-32,57.0,32
289128,57L-33,57.0,33
289129,57L-34,57.0,34
289130,57L-35,57.0,35
...,...,...,...
369886,51L-89,51.0,89
369888,51L-110,51.0,110
374594,73L-73,73.0,73
375276,13L-48,13.0,48


In [24]:
# using poles[transmissi] columns, add tml 'CONDUCTOR' 'STRUCTURES' 'LOOP/RADIAL' 'VOLTAGE kV' 'YEAR BUILT' to poles
example = 3
tml[tml['LINE #']==3]

,AREA,REGION,LINE #,LOCATION FROM,LOCATION TO,LENGTH\nKM,CONDUCTOR,STRUCTURES,LOOP/RADIAL,VOLTAGE kV,YEAR BUILT,*REBUILD\nYEAR,Age
0,1,STJ,3,Petty Harbour (1),Point A (str 13),0.9,4/0 AASC STAND,S POLE (STEEL),Radial,33,2002,2060,23
1,1,STJ,3,Point A (str 13),Point B (str 61),3.4,4/0 AASC STAND,S POLE (STEEL),Radial,33,2004,2062,21
2,1,STJ,3,Point B (str 61),Goulds (82),1.3,4/0 AASC STAND,S POLE (WOOD),Radial,33,2004,2062,21


In [25]:
def v_fromnum(row, col):
    string = row[col]
    if not isinstance(string,str):
        print(type(string))
    else:
        extract = string#.split("(")[1]
        try:
            number = int(''.join([char for char in extract if char.isdigit()]))
        except:
            number = ""
    return number


tml['from'] = tml.apply(v_fromnum, col = 'LOCATION FROM', axis = 1)
tml['to'] = tml.apply(v_fromnum, col = 'LOCATION TO', axis = 1)

tml[['from','to']]

,from,to
0,1,13
1,13,61
2,61,82
3,1,47
4,47,191
...,...,...
320,44,47
321,1,94
322,95,109
323,25,55


In [26]:
def v_findinfo(row):
    line = row['Transmissi-pre']
    des = {'CONDUCTOR': "", 'STRUCTURES': "", 'LOOP/RADIAL': "", 'VOLTAGE kV': "", 'YEAR BUILT': ""}
    if np.isnan(line):
        return des
    try:
        line = int(row['Transmissi-pre'])
        section = row['Transmissi-sub']
        mat = tml[tml['LINE #'] == line]
        firstrow = mat.iloc[0]
        for key in des.keys():
            des[key] = firstrow[key]
    except ValueError:
        print(f"{row['Transmission-pre']} invalid")
    return des

testrow = poles.dropna(subset = ['Transmissi']).iloc[0]
v_findinfo(testrow)

{'CONDUCTOR': '350 CBLE NONSTD',
 'STRUCTURES': 'U/G CABLE',
 'LOOP/RADIAL': 'Loop',
 'VOLTAGE kV': np.int64(66),
 'YEAR BUILT': np.int64(1966)}

In [27]:
poles['attempt'] = poles.apply(v_findinfo, axis=1)
poles[["CONDUCTOR", "STRUCTURES", "LOOP/RADIAL", "VOLTAGE kV", "YEAR BUILT"]] = pd.DataFrame(poles.attempt.tolist(), index=poles.index)
poles = poles.drop(['attempt','Transmissi-pre','Transmissi-sub'], axis = 1)


In [28]:
poles.CONDUCTOR.value_counts()

CONDUCTOR
                     366829
397.5 ACSR STAND       2348
477 ASC STAND          1916
4/0 AASC STAND         1582
715.5 ASC STAND         940
599.5 AASC STAND        760
559.5 AASC STAND        708
266.8 ACSR STAND        451
477 ASC                 353
244.4 ACSR NONSTD       333
397.5 ASCR STAND        205
1000 CBLE NONSTD         61
350 CBLE NONSTD          60
477 ASC 'COSMOS'         33
Name: count, dtype: int64

In [29]:
poles

,Unnamed: 0,X,Y,Z,fid,OBJECTID,CreationUs,DateCreate,LastUser,DateModifi,...,Region.1,Nearest coast (km),"Nearest coast (lat,lon)",Nearest coast (lat),Nearest coast (lon),CONDUCTOR,STRUCTURES,LOOP/RADIAL,VOLTAGE kV,YEAR BUILT
0,0,-53.875632,47.283649,99,1,296592,System,2018-03-04,tdbutt,2022-05-30,...,Avalon,1.839371,"[47.269877023705774, -53.86216104389701]",47.269877,-53.862161,,,,,
1,1,-53.870052,47.287196,96,2,296593,System,2018-03-04,tdbutt,2022-05-30,...,Avalon,2.166641,"[47.30158338776317, -53.889372823896686]",47.301583,-53.889373,,,,,
2,2,-53.869003,47.288435,93,3,296594,System,2018-03-04,tdbutt,2022-05-30,...,Avalon,1.907537,"[47.30395940470282, -53.8797431622171]",47.303959,-53.879743,,,,,
3,3,-53.866783,47.291068,89,4,296595,System,2018-03-04,tdbutt,2022-05-30,...,Avalon,1.611583,"[47.304732712844505, -53.873895296330836]",47.304733,-53.873895,,,,,
4,4,-53.865612,47.292564,92,5,296596,System,2018-03-04,tdbutt,2022-05-30,...,Avalon,1.398151,"[47.304891739961874, -53.869267832044784]",47.304892,-53.869268,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376574,376574,-53.533930,47.407960,60,376575,8533988,System,2025-10-03,System,2025-10-03,...,Avalon,12.296229,"[47.517491111026, -53.5565241109989]",47.517491,-53.556524,,,,,
376575,376575,-53.534330,47.407720,60,376576,8533989,System,2025-10-03,System,2025-10-03,...,Avalon,12.318523,"[47.517491111026, -53.5565241109989]",47.517491,-53.556524,,,,,
376576,376576,-56.073140,49.493630,12,376577,8534388,System,2025-10-03,System,2025-10-03,...,Grand Falls,0.381451,"[49.493305563700865, -56.067898310358714]",49.493306,-56.067898,,,,,
376577,376577,-58.842950,48.182880,58,376578,8534788,System,2025-10-03,System,2025-10-03,...,Stephenville,3.583077,"[48.19875978453649, -58.88488409882206]",48.198760,-58.884884,,,,,


In [30]:
if export:= True:
    import os
    if filter: name = "Poles-Tr-filt.csv"
    if not filter: name = "Poles-Tr-all.csv"
    folder = r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\NF Power GIS\Poles-tr"
    poles.to_csv(os.path.join(folder,name))
    print(f"{name} saved to {folder}")

Poles-Tr-all.csv saved to C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\NF Power GIS\Poles-tr
